## Imports & Config

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from sqlalchemy import create_engine
import names
import pandas as pd
import dask.dataframe as dd
import random
POSTGRESQL_URL = os.getenv("POSTGRESQL_URL")
if POSTGRESQL_URL is None:
    raise ValueError("POSTGRESQL_URL environment variable not set")
engine = create_engine(POSTGRESQL_URL)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
books_df = dd.read_parquet('../data/reduced_books.parquet')
interactions_df = dd.read_parquet('../data/reduced_interactions.parquet')
reviews_df = dd.read_parquet('../data/reduced_reviews.parquet')
authors_df = dd.read_parquet('../data/new_authors.parquet')
users = pd.read_csv("../data/user_id_map.csv")



### Books

In [4]:
books_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,"[2285777, 5941079, 3134684]",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,[fb1de694-168f-4a44-b604-34169f4cbf0e],Atria Books,368,14,9780743294294,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
15,0800759494,2885,[],US,,"[{'count': '9381', 'name': 'to-read'}, {'count...",,false,3.91,B00B853QPM,"[53817, 254389, 8964, 8139321]",As he is driving home from a minister's confer...,,https://www.goodreads.com/book/show/89375.90_M...,"[e1cead2a-37b0-4231-b350-1d6cf5a5c07f, 8c38df9...",,,,9780800759490,,,,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,89375,68157,2957021,90 Minutes in Heaven: A True Story of Death an...,90 Minutes in Heaven: A True Story of Death an...
479,,346,[274178],US,en-GB,"[{'count': '6001', 'name': 'to-read'}, {'count...",B0055Q8HDG,true,3.86,B0055Q8HDG,"[10950666, 11948797, 12711899, 11187203, 12901...",Sherry has always known there was something ou...,Kindle Edition,https://www.goodreads.com/book/show/11731782-c...,[b45ea77a-77e3-4efc-a8ae-0a8de1007f59],,360,12,,6,,2011,https://www.goodreads.com/book/show/11731782-c...,https://images.gr-assets.com/books/1352764436m...,11731782,5125,16680541,"Collide (Collide, #1)","Collide (Collide, #1)"
583,0395083621,1396,[],US,en-US,"[{'count': '549', 'name': 'history'}, {'count'...",,false,3.13,B008QI6EHQ,[3831344],"Madman, tyrant, animal - history has given Ado...",,https://www.goodreads.com/book/show/54270.Mein...,[caafdcb9-33c8-4284-9df3-7c2d27a2ef8b],Houghton Mifflin Company,384,,9780395083628,,,,https://www.goodreads.com/book/show/54270.Mein...,https://images.gr-assets.com/books/1395618385m...,54270,16837,2049624,Mein Kampf,Mein Kampf
807,0060773758,1223,[163036],US,en-US,"[{'count': '7188', 'name': 'to-read'}, {'count...",,false,3.91,B000GCFWXW,"[225669, 50789, 46481, 780878]",That hot guy tied to Lissianna Argeneau's bed?...,Mass Market Paperback,https://www.goodreads.com/book/show/38568.A_Qu...,[3f4997a5-de3c-45d0-932b-ce8540afcccc],Avon,360,25,9780060773755,10,,2005,https://www.goodreads.com/book/show/38568.A_Qu...,https://images.gr-assets.com/books/1410129015m...,38568,32140,2621331,A Quick Bite (Argeneau #1),A Quick Bite (Argeneau #1)


### Interactions

In [10]:
interactions_df.head()

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,6480781,c8676124d8829874576fcb868af89315,True,5,,Mon Mar 20 23:58:16 -0700 2017,Wed Mar 22 11:47:49 -0700 2017,,
1,8842281e1d1347389f2ab93d60773d4d,29584452,dacadc8f32e80bbdb5cd052a84ad5c63,True,4,,Tue Nov 15 09:23:32 -0800 2016,Tue Dec 13 11:00:02 -0800 2016,Wed Nov 23 00:00:00 -0800 2016,Fri Nov 18 00:00:00 -0800 2016
2,8842281e1d1347389f2ab93d60773d4d,28119237,7a8dc8ab7f3c0084be8150d7e5bd40cb,True,4,A fascinating book about community and belongi...,Thu Sep 22 16:19:12 -0700 2016,Thu Sep 22 16:19:12 -0700 2016,Tue Nov 22 00:00:00 -0800 2016,
3,8842281e1d1347389f2ab93d60773d4d,186074,1d12addadc0c737dcd29c362c936a266,True,5,,Tue Jul 12 19:21:10 -0700 2016,Wed Mar 22 11:47:46 -0700 2017,,
4,8842281e1d1347389f2ab93d60773d4d,15839976,c7baa53f6f7d554ed9c3859f0d400d19,True,5,,Tue May 10 11:34:44 -0700 2016,Wed Mar 22 11:47:45 -0700 2017,,


### Reviews

In [11]:
reviews_df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
10,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0
12,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
13,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,,,5,1
15,8842281e1d1347389f2ab93d60773d4d,32283133,8e4d61801907e591018bdc3442a9cf2b,0,http://www.telegraph.co.uk/culture/10...,Tue Nov 01 11:09:18 -0700 2016,Tue Nov 01 11:09:44 -0700 2016,,,9,0
17,8842281e1d1347389f2ab93d60773d4d,28119237,7a8dc8ab7f3c0084be8150d7e5bd40cb,4,A fascinating book about community and belongi...,Thu Sep 22 16:19:12 -0700 2016,Thu Sep 22 16:19:12 -0700 2016,Tue Nov 22 00:00:00 -0800 2016,,30,2


## Users

In [8]:
users.head()

,user_id_csv,user_id
0,0,8842281e1d1347389f2ab93d60773d4d
1,1,72fb0d0087d28c832f15776b0d936598
2,2,ab2923b738ea3082f5f3efcbbfacb218
3,3,d986f354a045ffb91234e4af4d1b12fd
4,4,7504b2aee1ecb5b2872d3da381c6c91e


In [ ]:
# Generate single names
#first_name = names.get_first_name()  # e.g. "John"
#last_name = names.get_last_name()    # e.g. "Smith"
#full_name = names.get_full_name()    # e.g. "John Smith"

In [ ]:
# Generate fake data for each user
user_records = []
counter = 0
for _, user in users.iterrows():
    first = names.get_first_name()
    last = names.get_last_name()
    name = f"{first} {last}"
    email = f"{first.lower()}.{last.lower()}{counter:05d}@example.com"    
    user_records.append({
        'id': user['user_id'],
        'name': name,
        'email': email,
    })
    counter += 1

# Create DataFrame and insert to DB
users_df = pd.DataFrame(user_records)
users_df.to_sql('users', engine, if_exists='append', index=False)


## Authors

In [3]:
authors_df = pd.read_parquet("../data/new_authors.parquet")
authors_df.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,f6fe6aac-2718-4a70-8482-5e32d24ce2b1,7,Ronald J. Fields,49
1,4.08,126cc433-0c42-4242-b721-9427dbfa74eb,28716,Anita Diamant,546796
2,3.92,550d8a5f-4893-47ec-8b77-ab82175410c4,5075,Barbara Hambly,122118
3,3.68,fb1de694-168f-4a44-b604-34169f4cbf0e,36262,Jennifer Weiner,888522
4,3.82,90bba374-d702-4a94-b16c-ebb6c00ffb1e,96,Nigel Pennick,1740


In [13]:
len(authors_df)

829529

In [12]:
# Generate fake emails for authors
author_records = []
counter = 713450
for _, author in authors_df.iterrows():
    name = author['name']
    if name == "":
        name = "No Name"
        email = f"noname{counter:06d}@example.com"  # Increased to 6 digits
    else:
        name_parts = name.lower().split()
        if len(name_parts) >= 2:
            email = f"{name_parts[0]}.{name_parts[-1]}{counter:06d}@example.com"
        else:
            email = f"{name_parts[0]}{counter:06d}@example.com"
        
    author_records.append({
        'id': author['author_id'],
        'name': name,
        'email': email,
        'is_author': True
    })
    counter += 1

# Create DataFrame and insert to DB
authors_df_with_email = pd.DataFrame(author_records)
authors_df_with_email.to_sql('users', engine, if_exists='append', index=False)


529

In [17]:
interactions_df = dd.read_parquet("../data/interactions.parquet")
interactions_df.head()

,user_id,book_id,is_read,rating,is_reviewed
0,8842281e1d1347389f2ab93d60773d4d,12,1,5,0
1,8842281e1d1347389f2ab93d60773d4d,21,1,5,1
2,8842281e1d1347389f2ab93d60773d4d,30,1,5,0
3,8842281e1d1347389f2ab93d60773d4d,45,1,5,0
4,8842281e1d1347389f2ab93d60773d4d,1,1,5,0


## Books

## BookAuthors

## BookGenres

## SimilarBooks

## Genres